# ERASTAR : Ouverture du datacube avec Dask 

Dans la partie précédente, nous avons créé un datacube, ici nous allons voir au moyen d'opérations simple, comment utiliser et configurer ce datacube pour optimiser sont utilisation avec Dask.

In [1]:
import os
import dask
import intake
import xarray as xr
import dask_jobqueue
import dask.distributed

from pathlib import Path

## Cluster Dask

In [2]:
# Création du cluster Dask
cluster = dask_jobqueue.PBSCluster(queue='mpi', 
                                   cores=28,
                                   memory="115GB",
                                   walltime="04:00:00",
                                   interface='ib0',
                                   local_directory='/tmp',
                                   log_directory='/home1/scratch/gcaer/dask-logs',
                                   #processes=,
                                  )
cluster.scale(jobs=2)

/home1/datawork/gcaer/conda-env/data-env/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 50395 instead
  warnings.warn(


In [3]:
client = dask.distributed.Client(cluster, timeout=600)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: http://10.148.1.123:50395/status,
Dashboard: http://10.148.1.123:50395/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.148.1.123:35518,Workers: 0
Dashboard: http://10.148.1.123:50395/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [4]:
port = client.scheduler_info()["services"]["dashboard"]
ssh_command = f'ssh -N -L {port}:{os.environ["HOSTNAME"]}:{port} {os.environ["USER"]}@datarmor.ifremer.fr'

print(f"{ssh_command}")
print(f"open browser at address of the type: http://localhost:{port}")

ssh -N -L 50395:r2i2n25:50395 gcaer@datarmor.ifremer.fr
open browser at address of the type: http://localhost:50395


## Ouverture du datacube

In [5]:
# Chemin vers le datacube
wrk = Path('/home/datawork-data-terra/odatis/data')
name = 'erastar'
version = 'datacube-year'
output = wrk / name / version

In [6]:
def open_datacube(output, chunks, start, end):
    """Ouverture du datacube"""
    vars = ["es_u10s", "es_v10s", "e5_u10s", "e5_v10s", "count", "quality_flag"]
    cat = intake.open_catalog(output / "reference.yaml")
    datacubes = [cat[str(year)](chunks=chunks).to_dask().drop_vars(vars) for year in range(start, end+1)]
    datacube = xr.concat(datacubes, dim="time")
    return datacube

Nous allons commencer par ouvrir le datacube sans préciser la taille des chunks.

In [22]:
%%time
chunks = {}
start, end = 2015, 2017
datacube = open_datacube(output, chunks, start, end)

/home1/datawork/gcaer/conda-env/data-env/lib/python3.11/site-packages/intake_xarray/base.py:21: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(self._ds.dims),
/home1/datawork/gcaer/conda-env/data-env/lib/python3.11/site-packages/intake_xarray/base.py:21: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(self._ds.dims),


CPU times: user 1.38 s, sys: 112 ms, total: 1.5 s
Wall time: 1.49 s


/home1/datawork/gcaer/conda-env/data-env/lib/python3.11/site-packages/intake_xarray/base.py:21: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(self._ds.dims),


In [23]:
datacube

<xarray.Dataset>
Dimensions:  (time: 26304, lat: 1440, lon: 2880)
Coordinates:
  * lat      (lat) float64 -89.94 -89.81 -89.69 -89.56 ... 89.69 89.81 89.94
  * lon      (lon) float64 -179.9 -179.8 -179.7 -179.6 ... 179.7 179.8 179.9
  * time     (time) datetime64[ns] 2015-01-01 ... 2017-12-31T23:00:00
Data variables:
    e5_tauu  (time, lat, lon) float32 dask.array<chunksize=(1, 1440, 2880), meta=np.ndarray>
    e5_tauv  (time, lat, lon) float32 dask.array<chunksize=(1, 1440, 2880), meta=np.ndarray>
    es_tauu  (time, lat, lon) float32 dask.array<chunksize=(1, 1440, 2880), meta=np.ndarray>
    es_tauv  (time, lat, lon) float32 dask.array<chunksize=(1, 1440, 2880), meta=np.ndarray>
Attributes: (12/55)
    Conventions:                 CF-1.7, ACDD-1.3, ISO 8601
    Metadata_Conventions:        Climate and Forecast (CF) 1.7, Attribute Con...
    acknowledgement:             Please acknowledge the use of these data wit...
    band:                        C, C
    cdm_data_type:               grid
    comment:                     These data were produced at ICM/CSIC as part...
    ...                          ...
    summary:                     This data set contains the stress equivalent...
    technical_support_contact:   cersat@ifremer.fr
    time_coverage_end:           20150101T00:00:00
    time_coverage_start:         20150101T00:00:00
    title:                       hourly stress equivalent model wind componen...
    track_id:                    54df782f-d688-4680-92c8-e198021e2d0d

Les chunks par défaut sont de la taille: `{"time": 1, "latitude": 1440, "longitude": 2880}`. On reconnait ici les chunks natif des fichiers netCDF.

## Task Graph

Nous regardons maintenant le task-graph correspondant au chargement de ce datacube. 

In [13]:
dask.delayed(datacube).dask

HighLevelGraph with 30 layers.
<dask.highlevelgraph.HighLevelGraph object at 0x2aabe63a5110>
 0. original-open_dataset-es_tauv-92e794d8904c6e84282faa9323fefa8c
 1. open_dataset-es_tauv-92e794d8904c6e84282faa9323fefa8c
 2. original-open_dataset-es_tauv-4b5e865f4febfe5e853935953e0e2236
 3. open_dataset-es_tauv-4b5e865f4febfe5e853935953e0e2236
 4. original-open_dataset-es_tauv-583b1913ebefb688a679d9134093bbdc
 5. open_dataset-es_tauv-583b1913ebefb688a679d9134093bbdc
 6. concatenate-c1e00fef91f9fd1d96c191ab50ea1d0e
 7. original-open_dataset-es_tauu-92e794d8904c6e84282faa9323fefa8c
 8. open_dataset-es_tauu-92e794d8904c6e84282faa9323fefa8c
 9. original-open_dataset-es_tauu-4b5e865f4febfe5e853935953e0e2236
 10. open_dataset-es_tauu-4b5e865f4febfe5e853935953e0e2236
 11. original-open_dataset-es_tauu-583b1913ebefb688a679d9134093bbdc
 12. open_dataset-es_tauu-583b1913ebefb688a679d9134093bbdc
 13. concatenate-e67c2c119a4c2b85dcadc7afd637dcc8
 14. original-open_dataset-e5_tauv-92e794d8904c6e84282faa9323fefa8c
 15. open_dataset-e5_tauv-92e794d8904c6e84282faa9323fefa8c
 16. original-open_dataset-e5_tauv-4b5e865f4febfe5e853935953e0e2236
 17. open_dataset-e5_tauv-4b5e865f4febfe5e853935953e0e2236
 18. original-open_dataset-e5_tauv-583b1913ebefb688a679d9134093bbdc
 19. open_dataset-e5_tauv-583b1913ebefb688a679d9134093bbdc
 20. concatenate-3986947ad47c4c87a23f1971f2a8c3b4
 21. original-open_dataset-e5_tauu-92e794d8904c6e84282faa9323fefa8c
 22. open_dataset-e5_tauu-92e794d8904c6e84282faa9323fefa8c
 23. original-open_dataset-e5_tauu-4b5e865f4febfe5e853935953e0e2236
 24. open_dataset-e5_tauu-4b5e865f4febfe5e853935953e0e2236
 25. original-open_dataset-e5_tauu-583b1913ebefb688a679d9134093bbdc
 26. open_dataset-e5_tauu-583b1913ebefb688a679d9134093bbdc
 27. concatenate-5dcbd234a7139d921079546801344330
 28. finalize-e179663c-d2a7-4d62-906f-aa812f5555ea
 29. Dataset-57b24098-e3a4-403e-81f6-57b0ad0e878e

Nous voyons que le chargement complet de ce datacube nécessite **30 layers** avec **210 446 tasks**. Il s'agit donc d'un gros task-graph, il serait donc intéréssant de voir comment Dask se comporte si on ne sélectionne qu'une tranche du datacube. 

In [24]:
%%time
result = datacube.isel(time=slice(0,12)).compute()

CPU times: user 1.12 s, sys: 1.06 s, total: 2.18 s
Wall time: 3.26 s


Nous obtenons le task-graph suivants:
  
<img src="./images/datacube-chunks-natif.png">

- `original`: Nous avons 4 tasks `original`, ces tasks correspondent à l'accès aux jeux de données, ici nous en avons 4 car nous avons 4 variables dans notre datacube (e5_tauu, e5_tauv, es_tauu, es_tauv, en effet Dask procède chaque variable de manière parallèle).
- `getitem`: Nous avons 48 tasks `getitem`, en effet si on regarde le nombre de chunks pour la variable `e5_tauu` sur la période `time=slice(0,12))` nous avons **12 chunks**. Comme nous avons 4 variables, nous avons donc 4 * 12 chunks ouvert et pour chacun d'entre eux nous récupérons toutes les données.

```Python
datacube.isel(time=slice(0,12)).e5_tauu.data
```

<img src="./images/datacube-isel-chunks.png">

La 1ère chose que nous voyons ici c'est que notre opération n'a pas nécessité l'éxecution des **210 446 tasks**, mais nous avons drastiquement réduit le nombre de tasks executé. En effet Dask ne s'intérèsse ici qu'au chunks qui sont concerné par notre opération. 

La 2ème chose, est que nous pouvons encore diminuer le nombre de tasks pour celà nous allons redéfinir la taille des chunks lors de la lecture du datacube (ils s'agit de chunks virtuels, mais qui permettent d'optimiser le fonctionnement de dask). 

## Optimisation de la taille des chunks

In [26]:
%%time
chunks = {"time": 6, "latitude": -1, "longitude": -1}
datacube = open_datacube(output, chunks, start, end)

/home1/datawork/gcaer/conda-env/data-env/lib/python3.11/site-packages/intake_xarray/base.py:21: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(self._ds.dims),
/home1/datawork/gcaer/conda-env/data-env/lib/python3.11/site-packages/intake_xarray/base.py:21: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(self._ds.dims),


CPU times: user 1.34 s, sys: 116 ms, total: 1.45 s
Wall time: 1.45 s


/home1/datawork/gcaer/conda-env/data-env/lib/python3.11/site-packages/intake_xarray/base.py:21: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(self._ds.dims),


In [27]:
dask.delayed(datacube).dask

HighLevelGraph with 30 layers.
<dask.highlevelgraph.HighLevelGraph object at 0x2aac4126d110>
 0. original-open_dataset-es_tauv-6e4468c15ef36bfca0863863d6ca9d99
 1. open_dataset-es_tauv-6e4468c15ef36bfca0863863d6ca9d99
 2. original-open_dataset-es_tauv-7408dc9a6965ba35a2bb4084eedf4038
 3. open_dataset-es_tauv-7408dc9a6965ba35a2bb4084eedf4038
 4. original-open_dataset-es_tauv-95b6265ae7ebcd29481bfa094b60f7f7
 5. open_dataset-es_tauv-95b6265ae7ebcd29481bfa094b60f7f7
 6. concatenate-1bec79754867c80f645aeb2f0c3e4fa8
 7. original-open_dataset-es_tauu-6e4468c15ef36bfca0863863d6ca9d99
 8. open_dataset-es_tauu-6e4468c15ef36bfca0863863d6ca9d99
 9. original-open_dataset-es_tauu-7408dc9a6965ba35a2bb4084eedf4038
 10. open_dataset-es_tauu-7408dc9a6965ba35a2bb4084eedf4038
 11. original-open_dataset-es_tauu-95b6265ae7ebcd29481bfa094b60f7f7
 12. open_dataset-es_tauu-95b6265ae7ebcd29481bfa094b60f7f7
 13. concatenate-c663271aa38736da2d3a3fa6047fdd32
 14. original-open_dataset-e5_tauv-6e4468c15ef36bfca0863863d6ca9d99
 15. open_dataset-e5_tauv-6e4468c15ef36bfca0863863d6ca9d99
 16. original-open_dataset-e5_tauv-7408dc9a6965ba35a2bb4084eedf4038
 17. open_dataset-e5_tauv-7408dc9a6965ba35a2bb4084eedf4038
 18. original-open_dataset-e5_tauv-95b6265ae7ebcd29481bfa094b60f7f7
 19. open_dataset-e5_tauv-95b6265ae7ebcd29481bfa094b60f7f7
 20. concatenate-6627f53439e93b61e5ca4d0435263839
 21. original-open_dataset-e5_tauu-6e4468c15ef36bfca0863863d6ca9d99
 22. open_dataset-e5_tauu-6e4468c15ef36bfca0863863d6ca9d99
 23. original-open_dataset-e5_tauu-7408dc9a6965ba35a2bb4084eedf4038
 24. open_dataset-e5_tauu-7408dc9a6965ba35a2bb4084eedf4038
 25. original-open_dataset-e5_tauu-95b6265ae7ebcd29481bfa094b60f7f7
 26. open_dataset-e5_tauu-95b6265ae7ebcd29481bfa094b60f7f7
 27. concatenate-47f0fa9ffd17f692b618af0a846d2a83
 28. finalize-63ba5ddd-8de4-4775-b470-b5dbe655e417
 29. Dataset-237b4fbe-1862-4508-b060-b10c524b0db0

Le task graph du chargement complet de ce datacube est passé de 30 layers avec 210 446 tasks à **30 layers** avec **35086 tasks**. Nous avons donc considérablement réduit la taille du task-graph. Si on s'intérèsse à nouveau à la sélection d'une tranche, nous obtenons les résultats suivants:

In [29]:
%%time
result = datacube.isel(time=slice(0,12)).compute()

CPU times: user 964 ms, sys: 1.59 s, total: 2.56 s
Wall time: 2.83 s


Nous obtenons le task-graph suivants:

<img src="./images/datacube-chunks-opti.png">

- `original`: Nous avons toujours 4 tasks `original`, ce qui est normal car nous avons toujours 4 variables dans notre datacube. 
- `getitem`: Nous n'avons plus que 8 tasks `getitem`, car nous n'avons plus que 4 * 2 chunks ouvert et pour chacun d'entre eux nous récupérons toutes les données. 

On vois ici l'importance de définir des chunks avec une taille correcte, en plus du nombre de tasks, la taille des chunks joue sur le temps d'éxectution. La taille des chunks n'est donc pas à négliger. 